In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/ITA Data CY 2021 submitted thru 3-14-2022 v3.csv', engine = 'python')

In [ ]:
df = df.drop(['id', 'zip_code', 'ein', 'industry_description', 'street_address', 'change_reason', 'delete', 'establishment_id', 'created_timestamp'], axis = 1 )
df = df.drop_duplicates()

In [ ]:
df

,company_name,establishment_name,city,state,naics_code,annual_average_employees,total_hours_worked,no_injuries_illnesses,total_deaths,total_dafw_cases,...,total_djtr_days,total_injuries,total_poisonings,total_respiratory_conditions,total_skin_disorders,total_hearing_loss,total_other_illnesses,establishment_type,size,year_filing_for
0,"Aza Services, LLC",Aza Services LLC,Jefferson,GA,238140,12.0,700.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2021.0
1,All Steel Welding LLC,All Steel Welding LLC,Channelview,TX,332312,20.0,38400.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2021.0
2,"CJF Shipping, LLC","CJF Shipping, LLC",Newark,NJ,492110,25.0,700.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2021.0
3,Sidebotham Substation Services PLLC,Main Office,Eastover,NC,221122,1.0,2080.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2021.0
4,I Do Furniture INC,Duckjung Kim,Honolulu,HI,236118,4.0,10800.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287023,Thombert Inc,Thombert,Brooklyn,IA,325211,48.0,89278.0,1.0,0.0,0.0,...,0.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2021.0
287024,ADF International,ADF International,Great Falls,MT,332312,144.0,290713.0,1.0,0.0,3.0,...,398.0,12.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2021.0
287025,TDK Corporation of America,TCA - San Jose,San Jose,CA,423690,33.0,63009.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2021.0
287026,TDK Corporation of America,TCA Lincolnshire,Lincolnshire,IL,423690,74.0,140121.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2021.0


In [ ]:
def feature_engineering(df):

    df = df.rename(columns={'company_name':'회사이름', 'establishment_name':'시설이름', 'establishment_name':'시설이름', 'city':'City',
                            'state':'State', 'naics_code':'북미산업코드', 'annual_average_employees':'AAE', 'total_hours_worked':'THW', 
                            'no_injuries_illnesses':'Injuries', 'establishment_type':'운영기관', 'size':'Size', 'year_filing_for':'Year',
                            'total_deaths':'총사망자', 'total_dafw_cases':'결근발생사례건수', 'total_djtr_cases':'이직과제한발생사례건수', 'total_other_cases':'기타근로불가발생사례건수',
                            'total_injuries':'총부상자', 'total_skin_disorders':'총피부병질환자', 'total_respiratory_conditions':'총호흡기질환자', 'total_poisonings':'총감염성질환자', 
                            'total_hearing_loss':'총난청질환자', 'total_other_illnesses':'기타질병질환자'})

    null_tar = df[(df['Injuries'].isnull())]
    df = df.drop(index = null_tar.index).reset_index(drop=True)

    error_employees = df[(df['AAE'] == 0)]
    df = df.drop(index = error_employees.index).reset_index(drop=True)
    error_hours = df[(df['THW'] == 0)]
    df = df.drop(index = error_hours.index).reset_index(drop=True)

    df['운영기관'] = df['운영기관'].fillna(1.0)

    df['북미산업코드'] = df['북미산업코드'].astype('str')
    df['산업코드'] = df['북미산업코드'].str[:2]
    naics = ['11', '21', '22', '23', '31', '32', '33', '42', '44', '45', '48', '49', '51', '52', '53', '54', '55', '56', '61', '62', '71', '72', '81', '92']
    df = df[df['산업코드'].isin(naics)]
    df['산업코드'] = df['산업코드'].astype('object')      
    name_mapping = {'11': 'AFFH', '21': 'MQOG', 
                    '22': 'UT', '23': 'CS', '31': 'MF', '32': 'MF','33': 'MF','42': 'WT',
                    '44': 'RT', '45': 'RT', '48': 'TW', '49': 'TW', 
                    '51': 'IF', '52': 'FI', '53': 'RE, RL','54': 'PST',
                    '55': 'MCE', '56': 'ASWMRS',
                    '61': 'ES', '62': 'HCSA', '71': 'AER', '72': 'AFS', 
                    '81': 'OS(NOT PA)', '92': 'PA'}
    func = lambda x: name_mapping.get(x, x)
    df['Industry'] = df.산업코드.map(func)

    df['Injuries'] = np.where(df['Injuries'] == 1.0, 1, 0) 

    df['회사이름'] = df['회사이름'].fillna('Unknown')
    df['시설이름'] = df['시설이름'].fillna('Unknown')
    df['City'] = df['City'].fillna('Unknown')

    df['State'] = df['State'].str.upper()

    df = df.replace({'운영기관' : 'Executive and Legislative Offices'}, 3)  
    df['Year'] = df['Year'].astype('int')
    df['Year'] = df['Year'].astype('object')

    Null_hours = df[(df['AAE'].isnull())]
    df = df.drop(index=Null_hours.index).reset_index(drop=True)

    df["EHW"] = round((df["THW"] / df["AAE"]), 0)

    outlier = df.loc[df['EHW'] > 4000, 'EHW'].index
    df = df.drop(labels=outlier, axis=0).reset_index(drop=True)

    df['Employee working hours'] = 0
    df['Employee working hours'] = pd.cut(df.EHW, bins=[0, 1000, 1200, 1400, 1600, 1800, 2000, 2400, 4000], labels=['1 ~ 1000 hr', '1000 ~ 1200 hr', '1200 ~ 1400 hr', '1400~ 1600 hr', '1600 ~ 1800 hr', '1800 ~ 2000 hr', '2000 ~ 2400 hr', '2400 hr ~'])

    outlier1 = df.loc[df['AAE'] > 2000, 'AAE'].index
    df = df.drop(labels=outlier1, axis=0).reset_index(drop=True)

    df['Average employees'] = 0
    df['Average employees'] = pd.cut(df.AAE, bins=[0 ,10, 25, 50, 150, 2000], labels=['1 ~ 10 employees', '10 ~ 25 employees', '25 ~ 50 employees', '50 ~ 150 employees', '150 employees ~ '])

    null_hhl = df[(df['Employee working hours'].isnull())]
    df = df.drop(index = null_hhl.index).reset_index(drop=True)

    df['Size'] = df['Size'].astype('int')
    size = [1, 2, 3]
    df = df[df['Size'].isin(size)]     
    name_mapping = {1: '20 employees under', 2: '20 ~ 249 employees', 3: '250 employees more'}
    func = lambda x: name_mapping.get(x, x)
    df['Company size'] = df.Size.map(func)
    df['Company size'] = df['Company size'].astype('object')

    df['운영기관'] = df['운영기관'].astype('int')
    ins = [1, 2, 3]
    df = df[df['운영기관'].isin(ins)]      
    name_mapping = {1: 'Not a government entity', 2: 'State Government entity', 3: 'Local Government entity'}
    func = lambda x: name_mapping.get(x, x)
    df['Institution'] = df.운영기관.map(func)
    df['Iperating institution'] = df['Institution'].astype('object')

    df["사망자발생여부"] = np.where(df["총사망자"].values > 0.0 , "1", "0" )
    df['사망자발생여부'] = df['사망자발생여부'].astype('object')
    df["결근발생여부"] = np.where(df["결근발생사례건수"].values > 0.0 , "1", "0" )
    df['결근발생여부'] = df['결근발생여부'].astype('object')
    df["이직과제한발생여부"] = np.where(df["이직과제한발생사례건수"].values > 0.0 , "1", "0" )
    df['이직과제한발생여부'] = df['이직과제한발생여부'].astype('object')
    df["기타근로불가발생여부"] = np.where(df["기타근로불가발생사례건수"].values > 0.0 , "1", "0" )
    df['기타근로불가발생여부'] = df['기타근로불가발생여부'].astype('object')
    df["부상자발생여부"] = np.where(df["총부상자"].values > 0.0 , "1", "0" )
    df['부상자발생여부'] = df['부상자발생여부'].astype('object')
    df["피부병질환발생여부"] = np.where(df["총피부병질환자"].values > 0.0 , "1", "0" )
    df['피부병질환발생여부'] = df['피부병질환발생여부'].astype('object')
    df["호흡기질환자발생여부"] = np.where(df["총호흡기질환자"].values > 0.0 , "1", "0" )
    df['호흡기질환자발생여부'] = df['호흡기질환자발생여부'].astype('object')
    df["감염성질환자발생여부"] = np.where(df["총감염성질환자"].values > 0.0 , "1", "0" )
    df['감염성질환자발생여부'] = df['감염성질환자발생여부'].astype('object')
    df["난청질환자발생여부"] = np.where(df["총난청질환자"].values > 0.0 , "1", "0" )
    df['난청질환자발생여부'] = df['난청질환자발생여부'].astype('object')
    df["기타질병질환자발생여부"] = np.where(df["기타질병질환자"].values > 0.0 , "1", "0" )
    df['기타질병질환자발생여부'] = df['기타질병질환자발생여부'].astype('object')

    df = df.drop(['회사이름', '시설이름', '북미산업코드', '산업코드', 'total_dafw_days', 'total_djtr_days', '총사망자', '결근발생사례건수', '이직과제한발생사례건수', '기타근로불가발생사례건수', '총부상자', '총피부병질환자', '총호흡기질환자', 
                  '총감염성질환자', '총난청질환자', '기타질병질환자', 'Iperating institution', '사망자발생여부', '결근발생여부', '이직과제한발생여부', '기타근로불가발생여부', '부상자발생여부',
                  '피부병질환발생여부', '호흡기질환자발생여부', '감염성질환자발생여부', '난청질환자발생여부', '기타질병질환자발생여부', 'Year', '운영기관', 'Institution', 'Size'], axis = 1 )


    df = df.drop_duplicates()
    df = df.reset_index(drop=True)

    return df

In [ ]:
df_clean = feature_engineering(df)

<ipython-input-4-d7b1274b0747>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['산업코드'] = df['산업코드'].astype('object')


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277013 entries, 0 to 277012
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   City                    277013 non-null  object  
 1   State                   277013 non-null  object  
 2   AAE                     277013 non-null  float64 
 3   THW                     277013 non-null  float64 
 4   Injuries                277013 non-null  int64   
 5   Industry                277013 non-null  object  
 6   EHW                     277013 non-null  float64 
 7   Employee working hours  277013 non-null  category
 8   Average employees       277013 non-null  category
 9   Company size            277013 non-null  object  
dtypes: category(2), float64(3), int64(1), object(4)
memory usage: 17.4+ MB


In [ ]:
df_cleaning = df_clean[(df_clean['Industry'] == 'CS') | (df_clean['Industry'] == 'MF') | (df_clean['Industry'] == 'TW')]
df_cleaning = df_cleaning.reset_index(drop=True)

In [ ]:
df_cleaning

,City,State,AAE,THW,Injuries,Industry,EHW,Employee working hours,Average employees,Company size
0,Jefferson,GA,12.0,700.0,0,CS,58.0,1 ~ 1000 hr,10 ~ 25 employees,20 employees under
1,Channelview,TX,20.0,38400.0,0,MF,1920.0,1800 ~ 2000 hr,10 ~ 25 employees,20 employees under
2,Newark,NJ,25.0,700.0,0,TW,28.0,1 ~ 1000 hr,10 ~ 25 employees,20 ~ 249 employees
3,Honolulu,HI,4.0,10800.0,0,CS,2700.0,2400 hr ~,1 ~ 10 employees,20 employees under
4,Gypsum,CO,14.0,23485.0,0,CS,1678.0,1600 ~ 1800 hr,10 ~ 25 employees,20 employees under
...,...,...,...,...,...,...,...,...,...,...
106298,Riverside,CA,66.0,87048.0,1,TW,1319.0,1200 ~ 1400 hr,50 ~ 150 employees,250 employees more
106299,Phoenix,AZ,28.0,54687.0,0,MF,1953.0,1800 ~ 2000 hr,25 ~ 50 employees,20 ~ 249 employees
106300,Brooklyn,IA,48.0,89278.0,1,MF,1860.0,1800 ~ 2000 hr,25 ~ 50 employees,20 ~ 249 employees
106301,Great Falls,MT,144.0,290713.0,1,MF,2019.0,2000 ~ 2400 hr,50 ~ 150 employees,20 ~ 249 employees


In [ ]:
df_cleaning.head()

,City,State,AAE,THW,Injuries,Industry,EHW,Employee working hours,Average employees,Company size
0,Jefferson,GA,12.0,700.0,0,CS,58.0,1 ~ 1000 hr,10 ~ 25 employees,20 employees under
1,Channelview,TX,20.0,38400.0,0,MF,1920.0,1800 ~ 2000 hr,10 ~ 25 employees,20 employees under
2,Newark,NJ,25.0,700.0,0,TW,28.0,1 ~ 1000 hr,10 ~ 25 employees,20 ~ 249 employees
3,Honolulu,HI,4.0,10800.0,0,CS,2700.0,2400 hr ~,1 ~ 10 employees,20 employees under
4,Gypsum,CO,14.0,23485.0,0,CS,1678.0,1600 ~ 1800 hr,10 ~ 25 employees,20 employees under


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_cleaning.to_csv('/content/drive/MyDrive/df_cleaning_2021.csv', index=False)